Copyright 2020-2022 Universidad Complutense de Madrid

This file software has been employed to calibrate optical spectroscopic data from the OSIRIS instrument at GTC (see [Paliya et al. 2020](https://ui.adsabs.harvard.edu/abs/2020ApJ...903L...8P/abstract))

Authors: Cristina Cabello (criscabe@ucm.es), Nicolás Cardiel (cardiel@ucm.es) Jesús Gallego (j.gallego@ucm.es)

SPDX-License-Identifier: GPL-3.0+ License-Filename: LICENSE.txt

In [3]:
import matplotlib.pyplot as plt

from astropy.io import fits
import numpy as np

# <span style="color:red"> OSIRIS calibrations: bias, flats, arcs</span>  

# 1) Bias
### - 1.0) MasterBias

```
$ cd bias/
$ numina-ximshow --extnum 3 *.fits
```

<span style="color:darkblue"> The individual bias images were visually inspected to ensure homogeneity and to check if they present a similar number of counts. </span>

<span style="color:darkblue"> The MasterBias image was computed by a median combination of the images, which enables removing many cosmic rays.</span>

```
$ ls 00025812*fits > list_bias.txt
$ numina-r6-imcombine list_bias.txt --method median --extnum 3 bias_median.fits

```

# 2) Flat
### - 2.0) MasterFlat

```
$ cd flat/
$ numina-ximshow --extnum 3 *.fits

```

<span style="color:darkblue"> The individual flat images were visually inspected to ensure homogeneity and to check if they present a similar number of counts. We combined them using the median.</span>

```
$ ls 000258136*fits > list_flats.txt
$ numina-r6-imcombine list_flats.txt --method median --extnum 3 flat_median.fits
```

<span style="color:darkblue"> We subtracted the MasterBias:</span>

```
$ numina-imath flat_median.fits - ../bias/bias_median.fits flat_median_bias.fits
```

<span style="color:darkblue"> In addition, in order to exclude the overscan region and to have the spectral direction in the X axis, we trimmed and rotated 90 degrees the resulting image. The MasterFlat image was computed by normalizing this flat image (i.e., dividing it by a median flat image).</span>


In [ ]:
fname = 'flat_median_bias.fits'
with fits.open(fname) as hdul:
    header = hdul[0].header
    data = hdul[0].data

# trim image
data = data[:, 25:950]

# rotate image
data = np.rot90(data, 1)
naxis2, naxis1 = data.shape

# compute median spectrum
spmedian = np.median(data, axis=0)
spmedian2d = np.tile(spmedian, naxis2).reshape(naxis2, naxis1)
data = data / spmedian2d

# save image
hdu = fits.PrimaryHDU(data, header=header)
hdulist = fits.HDUList(hdu)
outfile = 'flat_norm.fits'
hdulist.writeto(outfile, overwrite=True)

# 3) Arc
### - 3.0) Arc image

<span style="color:darkblue"> We had two different arcs, one taken with the Neon lamp and one with the HgAr lamp.</span>

```
$ fitsheader 000258136*fits -k object -f
                       filename                          OBJECT   
----------------------------------------------------- ------------
0002581363-20200526-OSIRIS-OsirisCalibrationLamp.fits ArcLamp_HgAr
0002581364-20200526-OSIRIS-OsirisCalibrationLamp.fits   ArcLamp_Ne
````

<span style="color:darkblue"> These two images were combined using the mean and then we subtracted the MasterBias. </span>

```
$ ls 000258136*fits > list_arcs.txt
$ numina-r6-imcombine list_arcs.txt --method mean --extnum 3 arc_mean.fits
$ numina-imath arc_mean.fits - ../bias/bias_median.fits arc_mean_bias.fits
````

<span style="color:darkblue"> Then, we trimmed, rotated and divided the result by the MasterFlat.</span>



In [ ]:
with fits.open('../flat/flat_norm.fits') as hdul:
    header = hdul[0].header
    flat_norm = hdul[0].data


fname = 'arc_mean_bias.fits'
with fits.open(fname) as hdul:
    header = hdul[0].header
    data = hdul[0].data
        
# trim image
data = data[:, 25:950]

# rotate image
data = np.rot90(data, 1)

# divide by flatfield
data = data / flat_norm

# save image
hdu = fits.PrimaryHDU(data, header=header)
hdulist = fits.HDUList(hdu)
outfile = 'arc_mean_bias_rf.fits'
hdulist.writeto(outfile, overwrite=True) 

<span style="color:darkblue"> We used the cleanest (https://cleanest.readthedocs.io/en/latest/) software [(Cardiel 2020)](https://ui.adsabs.harvard.edu/abs/2020ASPC..522..723C/abstract) to interpolate the hot and cold pixels of the resulting arc image:</span>

```
$ cleanest: arc_mean_bias_rf.fits -> arc_mean_bias_rfc.fits
````

<span style="color:darkblue"> We started to make use of the functionalities of the ReDucEme (https://reduceme.readthedocs.io/en/latest/) reduction package [(Cardiel 1999)](https://ui.adsabs.harvard.edu/abs/1999PhDT........12C/abstract).</span>

<span style="color:darkblue"> First, we converted the image from FITS format to REDUCEME format:</span>

```
$ R5-leefits_new: arc_mean_bias_rfc.fits -> arc_mean_bias_rfc.u

```


### - 3.1) Distortion C

<span style="color:darkblue"> We extracted an average spectrum from the 2D arc image and had a look at the shape of the arc lines.</span>

```
$ R5-adnsc

*******************************************************************************
REDUCEMEv5.0                   Welcome to adnsc       Version: 28-November-1996
-------------------------------------------------------------------------------

Work with error images (y/n) [y] ? n
Output into a single spectrum (y/n) [y] ? 
Input file name? arc_mean_bias_rfc.u
>>> NSCAN :    925
>>> NCHAN :   2051
>>> STWV  : 0.00000000
>>> DISP  : 0.00000000
>>> OBJECT: [not found]

* If you are using different factors for each spectrum, DO NOT normalize
Normalize final spectrum (y/n) [y] ? 
Scan region (0,0=EXIT) [0,0] ? 860,920
>>> Number of scans added......:     61
>>> Total number of scans added:     61
Scan region (0,0=EXIT) [0,0] ? 
Output file name? arc_mean_bias_rfc.usx

```

<span style="color:darkblue"> We found that the lines present C distortion, so we need to correct it.
We chose six isolated bright lines, and generated the file `lincdis.dat` with the X positions of these lines in a central scan (approx. 500):</span>

```
$ cat lincdis.dat
250
437
997
1698
1788
1870
```

<span style="color:darkblue"> The ‘fitcdis’ function allowed us to determine the distortion C of the arc image by polynomial fits to the brightest arc lines.</span>

```
$ R5-fitcdis
*******************************************************************************
REDUCEMEv5.0                  Welcome to fitcdis       Version: 6-December-1996
-------------------------------------------------------------------------------

File name with initial line positions [lincdis.dat] ? 
No. of line positions read:    6
Offset to be added to all the lines (integer) [0] ? 
No. of channels to search for line peaks (integer)  (1,...,9999) [3] ? 20
Arc file name? arc_mean_bias_rfc.u
>>> NSCAN :    925
>>> NCHAN :   2051
>>> STWV  : 0.00000000
>>> DISP  : 0.00000000
>>> OBJECT: [not found]
Spectra from... to... [1, 925] ? 
YRMSTOL (0=1.E-6) [0] ? 

Working...10%, 20%, 30%, 40%, 50%, 60%, 70%, 80%, 90%, 100% OK!

Output file name? arc.cdis1

********************************
FIT of line peak vs. scan number
********************************
Show fits for every line (y/n) [y] ? 
No. of plots in X [1] ? 3
No. of plots in Y [1] ? 2
Times sigma to exclude points [3] ? 
Graphic device #1 (? to see list) [/XServe] ? 
Graphic device #2 (NONE=EXIT) (? to see list) [NONE] ? 

FIT of line peak vs. scan number
Polynomial degree  (0,...,19) [2] ? 2
Line #  1 at channel # 250 -> no. excluded points:  7
Line #  1 at channel # 250 -> no. excluded points:  2
Line #  1 at channel # 250 -> no. excluded points:  0
Plot next line (y/n/g/p) [y] ? 
Line #  2 at channel # 437 -> no. excluded points:  1
Line #  2 at channel # 437 -> no. excluded points:  0
Plot next line (y/n/g/p) [y] ? 
Line #  3 at channel # 997 -> no. excluded points:  0
Plot next line (y/n/g/p) [y] ? 
Line #  4 at channel #1698 -> no. excluded points:  7
Line #  4 at channel #1698 -> no. excluded points:  1
Line #  4 at channel #1698 -> no. excluded points:  0
Plot next line (y/n/g/p) [y] ? 
Line #  5 at channel #1788 -> no. excluded points:  8
Line #  5 at channel #1788 -> no. excluded points:  4
Line #  5 at channel #1788 -> no. excluded points:  0
Plot next line (y/n/g/p) [y] ? 
Line #  6 at channel #1870 -> no. excluded points:  3
Line #  6 at channel #1870 -> no. excluded points:  0
Plot next line (y/n/g/p) [y] ? 

Normalization at scan no.  (1,...,925) [463] ? 

**********************************************
FIT of line peak deviations vs. channel number
**********************************************
Show fits for every scan:
(1=EACH 10 ,2=1ST AND LAST, 3=noplot) [1] ? 
Times sigma to exclude points [3] ? 
YMAX for this plot (ymax=-ymin) in channels [5.0] ? 30
 Type <RETURN> for next page: 

FIT of line peak deviations vs. channel number
Polynomial degree  (0,...,19) [2] ? 2

>>> SCAN #    1   MAX=22.98430   MEAN=********   SIG=15.74580
 Type <RETURN> for next page: 
 Type <RETURN> for next page: 
 ...
 ...
 Type <RETURN> for next page: 

>>> SCAN #  925   MAX= 9.64268   MEAN= 4.06479   SIG= 6.11200

```


<span style="color:darkblue"> We applied the derived C-distortion correction to the arc image using the ‘cdisc’ function:</span>

```
 $ R5-cdisc
*******************************************************************************
REDUCEMEv5.0                   Welcome to cdisc       Version: 28-November-1996
-------------------------------------------------------------------------------

Arc file name? arc_mean_bias_rfc.u
>>> NSCAN :    925
>>> NCHAN :   2051
>>> STWV  : 0.00000000
>>> DISP  : 0.00000000
>>> OBJECT: [not found]
Spectra from... to... [1, 925] ? 
(From fitcdis) FIT of line peak deviations vs. channel number:
Degree of fitted polynomial  (0,...,19)? 2
Polynomial file name (from fitcdis)? arc.cdis1
Output file name? arc.c1 
```

<span style="color:darkblue"> We used the xnirspec (https://github.com/nicocardiel/xnirspec) software [(Cardiel et al. 2003)](https://ui.adsabs.harvard.edu/abs/2003ApJ...584...76C/abstract) to inspect the resulting image and checked that now the arc lines are straight.</span>

```
$ xnirspec arc.c1
```


<img src="../plots_corrections/plots_finales/00_C_distortion.png" width="400px">


### - 3.2) Wavelength calibration


<span style="color:darkblue"> We extracted an average spectrum from the 2D arc image corrected for distortion C: </span>

```
$ R5-adnsc
*******************************************************************************
REDUCEMEv5.0                   Welcome to adnsc       Version: 28-November-1996
-------------------------------------------------------------------------------

Work with error images (y/n) [y] ? n
Output into a single spectrum (y/n) [y] ? 
Input file name? arc.c1
>>> NSCAN :    925
>>> NCHAN :   2051
>>> STWV  : 0.00000000
>>> DISP  : 0.00000000
>>> OBJECT: [not found]

* If you are using different factors for each spectrum, DO NOT normalize
Normalize final spectrum (y/n) [y] ? 
Scan region (0,0=EXIT) [0,0] ? 1,925
>>> Number of scans added......:    925
>>> Total number of scans added:    925
Scan region (0,0=EXIT) [0,0] ? 
Output file name? arc.c1sx

```


<span style="color:darkblue"> Then, we identified the lines in our spectrum by comparison with the expected arc lines given in the GTC website (`R1000B.ldat` file, see also http://www.gtc.iac.es/instruments/osiris/#Longslit_Spectroscopy). The line identification was carried out interactively thanks to the ‘findarc’ function.</span>

```
$ R5-findarc 
*******************************************************************************
REDUCEMEv5.0                  Welcome to findarc       Version: 6-December-1996
-------------------------------------------------------------------------------

Are you using graphic buttons.....(y/n) [y] ? 
Graphic device #1 (? to see list) [/XServe] ? 
Graphic device #2 (NONE=EXIT) (? to see list) [NONE] ? 
Spectrum file name? arc.c1sx
>>> NSCAN :      1
>>> NCHAN :   2051
>>> STWV  : 0.00000000
>>> DISP  : 0.00000000
>>> OBJECT: [not found]
Enter file with wavelenghts for current arc type [*.dat] ? R1000B.ldat
No. of line positions read:     47
No. of channels at each side to fit maximum  (1,...,9999) [2] ? 
Press cursor at two corners of the imaginary BOX to be zoomed
...
...
Output file name (save positions and wavelenghts) [fitlin.dat] ? 
No. of lines saved:     32
Do you really want to QUIT (y/n) [y] ? y
Do you want to see plots with the identified lines (y/n) [n] ? y
No. of channels:   2051
How many plots do you want to divide the spectrum into [5] ? 2
No. of plots in X,Y [1,1] ? 1,2
 Graphics device/type (? to see list, default /XServe): /cps
```


<img src="../plots_corrections/plots_finales/0_line_identification.png" width="800px">



<span style="color:darkblue"> We determined the wavelength calibration polynomial with the ‘fitlin’ function:  </span>

```
$ R5-fitlin
*******************************************************************************
REDUCEMEv5.0                   Welcome to fitlin       Version: 6-December-1996
-------------------------------------------------------------------------------

Graphic device #1 (? to see list) [/XServe] ? 
Graphic device #2 (NONE=EXIT) (? to see list) [NONE] ? fitlin.ps/cps
Graphic device #3 (NONE=EXIT) (? to see list) [NONE] ? 
Arc file name? arc.c1sx
>>> NSCAN :      1
>>> NCHAN :   2051
>>> STWV  : 0.00000000
>>> DISP  : 0.00000000
>>> OBJECT: [not found]
File name with line positions and wavelengths [fitlin.dat] ? 
Factor to be applied to wavelength[1.0] ? 
No. of lines read:     32
Do you want to reverse the line positions (y/n) [n] ? 
Centroid (Line=Line+Centroid) [0] ? 
Increment to search for peaks (0=no search)  (0,...,9999) [2] ? 
           1   3650.15332               5           5           0
           2   4046.56323             258         258           0
           3   4358.32812             437         437           0
           4   5460.73486             997         997           0
           5   5769.59814            1141        1141           0
           6   5790.66309            1151        1151           0
           7   5852.48779            1179        1179           0
           8   5881.89502            1192        1192           0
           9   5944.83398            1221        1221           0
          10   5975.53418            1235        1235           0
          11   6029.99707            1260        1260           0
          12   6074.33789            1279        1280           1
          13   6096.16309            1289        1289           0
          14   6143.06299            1310        1310           0
          15   6163.59424            1319        1320           1
          16   6217.28076            1343        1343           0
          17   6266.49512            1365        1365           0
          18   6304.78906            1382        1382           0
          19   6382.99219            1417        1417           0
          20   6402.24805            1425        1425           0
          21   6506.52783            1471        1471           0
          22   6532.88184            1483        1483           0
          23   6598.95312            1511        1511           0
          24   6678.27588            1546        1546           0
          25   6717.04297            1563        1563           0
          26   6929.46680            1654        1654           0
          27   7032.41309            1698        1698           0
          28   7173.93799            1758        1758           0
          29   7245.16748            1788        1789           1
          30   7438.89893            1870        1870           0
          31   7635.10596            1952        1952           0
          32   7724.20703            1989        1989           0

>>> Max. deviation:      1
>>> Mean:    9.37500000E-02

YRMSTOL for DOWNHILL [9.99999997E-07] ? 
How many points in each side of gaussian (maximum = 4) [3] ? 
           1           5           3           3           7
           2         258           3           3           7
           3         437           3           3           7
           4         997           3           3           7
           5        1141           3           3           7
           6        1151           3           3           7
           7        1179           3           3           7
           8        1192           3           3           7
           9        1221           3           3           7
          10        1235           3           3           7
          11        1260           3           3           7
          12        1280           3           3           7
          13        1289           3           3           7
          14        1310           3           3           7
          15        1320           3           3           7
          16        1343           3           3           7
          17        1365           3           3           7
          18        1382           3           3           7
          19        1417           3           3           7
          20        1425           3           3           7
          21        1471           3           3           7
          22        1483           3           3           7
          23        1511           3           3           7
          24        1546           3           3           7
          25        1563           3           3           7
          26        1654           3           3           7
          27        1698           3           3           7
          28        1758           3           3           7
          29        1789           3           3           7
          30        1870           3           3           7
          31        1952           3           3           7
          32        1989           3           3           7
Plot fits (-1=no, 0=range, I=line I) [-1] ? 
Fixed lines (0=exit)  (0,...,32) [0] ? 
Excluded lines (0=exit) [0] ? 
Polynomial degree  (0,...,19)? 7
>>> Coefficients of linear approximation (STWV,DISP):
   3440.98999       2.09249043    

>>> Coefficients of fitted polynomial:
           1   3643.13354    
           2   1.45365322    
           3   4.44676232E-04
           4  -5.04630222E-08
           5  -5.03223851E-11
           6   3.55521649E-14
           7  -1.00071541E-17
           8   1.03402201E-21

>>> Sigma:  0.018

Largest deviations:
Line # 27     Wavelength:    7032.4131     Deviation:       0.0596
Line #  5     Wavelength:    5769.5981     Deviation:       0.0459
Line # 31     Wavelength:    7635.1060     Deviation:       0.0259
Line # 28     Wavelength:    7173.9380     Deviation:       0.0254

Plots (y/n) [y] ? 

>>> PLOT 1: POINTS: differences between data and straigh line fit
            LINE  : differences between polynomial and straigh line

>>> PLOT 2: POINTS: differences between data and fit to a polynomial

Iterate (y/n) [y] ? n
Polynomial (output) file name? polarc

```

<img src="../plots_corrections/plots_finales/1_wavecalib.png" width="800px">



<span style="color:darkblue"> Finally, we used this polynomial and the ‘calambda’ function to obtain the parameters for the wavelength calibration, i.e. the wavelength as a function of the pixel number of a spectrum. </span>

```
$ R5-calambda 
*******************************************************************************
REDUCEMEv5.0                  Welcome to calambda     Version: 28-November-1996
-------------------------------------------------------------------------------

Polynomial file name? polarc
NCHAN? 2051
Pol. degree:            7
Pixel:     1  Lambda:    3644.58765    
Pixel:  2051  Lambda:    7872.55664    
Dispersion:    2.06242371    
Pixel (0=EXIT)  (0,...,2051) [0] ? 
```

<span style="color:darkblue"> We obtained a wavelength of 3644.59 
<span>&#8491;</span> for the first pixel and dispersion of 2.06 
<span>&#8491;</span>/px. </span>
